# GENDER AND STOCK MARKET PARTICIPATION
_Karl Stavem_

## Table of Contents
1.  [Motivation and Problem Statement](#motivation)
1.  [Background and Related Work](#brr)
1.  [Research Questions and Hypotheses](#rq)
1.  [Data Description](#dd)
1.  [Methodology](#methodology)
1.  [Data Preprocessing](#dpp)
1.  [Findings](#findings)
1.  [Discussion](#discussion)
1.  [Conclusion](#conclusion)

<span id="motivation"/>
    
### Motivation and Problem Statement OR INTRODUCTION



<span id="brr" />

### Background and Related Work

FINRA runs their own analysis of this dataset each time the survey is conducted.   Those results are available on [their website](https://www.usfinancialcapability.org/results.php?region=US).

<span id="dd" />

### Data


#### Descrption
The primary dataset used to address these questions is the _2018 National Financial Capability Study (NFCS)_, funded by the FINRA Investor Education Foundation.  The NFCS is a longitudinal survey conducted across the United States every three years.   The goal of the survey is to benchmark key indicators of financial capability in U.S. households and evaluate how these indictors vary by regions, attitudes, and demographics.  While this is a longitudinal survey that has been conducted every three years since 2009, my analysis focuses exclusively on the 2018 dataset.  While the FINRA Investor Education Foundation publishes its own findings on this data, it has made the datasets, questionaires, and documents available to outside researchers.

#### Dimensions
The 2018 NFCS contains responses from roughly 27,000 households across all 50 states.  Each respondent provided answers to 127 different questions.   All responses have been numericall encoded into a single .csv file.


#### Access

The 2018 NFCS dataset is freely available online and is subject to FINRA's [terms of use](https://www.usfinancialcapability.org/terms.php).  Full details and descrptions can be found on the on the [Data and Downloads](https://www.usfinancialcapability.org/downloads.php) page of the US Financial Capability website.  The primrary dataset used in this exploration is titled, _2018 State-by-State Survey — Respondent-Level Data, Comma delimited Excel file (.csv)_ and it may be downloaded in a .zip file directly from FINRA's website using [this link](https://www.usfinancialcapability.org/downloads/NFCS_2018_State_by_State_Data_Excel.zip).  This project already contains the full 2018 NFCS dataset, which can be viewed directly here: [\"NFCS 2018 State Data 190603.csv\"](raw_data/NFCS%202018%20State%20Data%20190603.csv). 

<span id="rq"/>

### Research Questions and Hypotheses

There are two primary questions that I will explore in this project:

1.  _R1:  Do women participate in the stock market at a lower rate than men?_


  
2.  _R2.1:  What are they key factors that affect stock market participation?_  

    
    _R2.2:  Do these key factors disproportionally affect women?_
        

My initial hypotheses were as follows:

_H1:  Women do participate in the stock market at lower rate than men.__
_H2:  The primary factors that influence stock market participation are related issues like income level and employment type, rather than perceptions or knowledge of the market.__
_H3:  Since women have a lower rate of full time employment, and on average earn less than their male 


<span id="methodology" />

### Methodology



<span id="dpp"/>

### Data Preprocessing

Here is the data preprocessing

In [2]:
import pandas as pd
df_sbs = pd.read_csv('raw_data/NFCS 2018 State Data 190603.csv')

<span id="findings" />
          
          
### Findings

asdfjk;lsadjfkjdsaf

#### R1:   Do Women Participate in the Stock Market at a Lower Rate Than Men?

dsafkj;asdfkjdsaf
          

#### R2: 1.   What are some of the factors that blah blah blah>

dsafkj;asdfkjdsaf

#### R2: 2. Do these factors disproportionally affect women?


dsafkj;asdfkjdsaf

<span id="discussion" />

### Discussion

<span id="conclusion" />

### Conclusion
blah blah blah

<span id="questions"/>

